In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ntu\anaconda3\envs\tensorflow-gpu\lib\site-

In [2]:
user_following = np.load('../Data/npy/user_following_1489.npy')
all_feature = np.load('../Data/npy/all_2939D_img0.5.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [4]:
video_f = np.load('../Data/npy/Video_512D_6650.npy')
text_f = np.load('../Data/npy/Text_300D.npy')
image_f = np.load('../Data/npy/Image_50_2048D.npy')
following_f = np.load('../Data/npy/Following_64D.npy')
social_f = np.load('../Data/npy/social_Norm15D.npy')
print('video feaute shape',video_f.shape)
print('text feature shape',text_f.shape)
print('image feature shape',image_f.shape)
print('following feature shape',following_f.shape)
print('social feature shape',social_f.shape)

video feaute shape (88, 512)
text feature shape (88, 300)
image feature shape (88, 2048)
following feature shape (88, 64)
social feature shape (88, 15)


In [5]:
#visual_feature = np.concatenate((video_feature, image_feature), axis=1)
#print('visual feature shape:',visual_feature.shape)
visual_feature = image_f
print('visual feature shape:',visual_feature.shape)

visual feature shape: (88, 2048)


In [6]:
user_test_amount = 150 #150 user for testing
yt_test_amount = 18 #18 YouTuber for testing for each tested user

In [7]:
def generate_train_test(user_following,feature,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(88):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(88):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
            f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm

In [8]:
train_t,train_f,test_t,test_f,user_category_norm = generate_train_test(user_following,visual_feature,user_category,YouTuber_category,active_users,user_test_amount,yt_test_amount)

Generate_train_test----
user_following shape  (1489, 88)
feature shape  (88, 2048)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)
user_category after normalized by max...
user_category_norm shape  (1489, 17)
Min number of followings  5
Max number of followings  34
test_idx [3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 732, 734, 739, 742, 749, 752, 761, 770, 779, 784, 797, 827, 831, 835, 849, 857, 863, 886, 911, 927, 933, 946, 947, 953, 960, 967, 984, 985, 990, 1049, 1050, 1074, 1085, 1092, 1104, 1113, 1116, 1124, 1175, 1184, 1200, 1207, 1216, 1220, 1230, 1235, 1250, 1264, 1265, 1275, 1277, 1281, 1283, 1307, 1329, 1333, 

In [9]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg) 

training 9.758226997985226
testing 5.466666666666667


In [10]:
all_auxilary = [i for i in range(88)]

# Recommendation Module

In [11]:
n = len(user_following)
m = 88  
k = 64
l = visual_feature.shape[1]
embedding_dims = 200

In [12]:
def training(save_name): 
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()
    
    train_yes_id=[] 
    for q in range(7):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            #train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                #pos = sample.index(ta)
                
                image_1=np.expand_dims(visual_feature[ta],0) #(1,2048)
                train_f_sample = random.sample(train_f[z],10)
                
                for b in train_f_sample:
                    image_2=np.expand_dims(visual_feature[b],0) #(1,2048)
                    _auc, _loss,_=sess.run([auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b],image_i:image_1,image_j:image_2})

                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1

        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, E,B =sess.run([user_latent, item_latent, embedding,Beta])
    np.savez('../Data/grid_search_weight/vbpr/'+save_name+'.npz', 
                        U=U, Y=Y, E=E,B=B)
    return U, Y, E, B

In [13]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]

testcount = 0
finish_list = []
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished Dims',finish_list)
                    finish_list.append(embedding_dims)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    """
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))
                    """


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        #aux_item = tf.get_variable("aux_item", [m, k],
                        #                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        """
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        """
                       
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    """
                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    """
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor

                    """
                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax
                    
                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]
                    
                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,64)
                    
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])
                    """

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """
                    
                    #aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    #aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,u) #把aux_new 的 值變成aux_np
                    
                    

                    """
                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    """
                    
                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),
                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr,Er,Br = training('vbpr_onlyImg')
                    print('Finish dims:,',embedding_dims)

Finished Dims []
Now Dims: 200
Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Iteraction: 0
total_loss:----------------- [[0.43166068]]
train_auc:------------------- 0.8432484514796972
time: 195.60016345977783  sec
Iteraction: 1
total_loss:----------------- [[0.35582776]]
train_auc:------------------- 0.8807432897453544
time: 391.0248763561249  sec
Iteraction: 2
total_loss:----------------- [[0.33395679]]
train_auc:------------------- 0.8905024088093599
time: 586.6089749336243  sec
Iteraction: 3
total_loss:----------------- [[0.31863308]]
train_auc:------------------- 0.8982105987611838
time: 782.7676899433136  sec
Iteraction: 4
total_